In [ ]:
'''This code is used to test a model trained using PYTHON_C_MIXING_v1.ipynb'''

In [ ]:
import numpy as np
import os
from math import pi, ceil, sqrt
import time
import pandas as pd
import pathlib
import ctypes
from IPython.display import clear_output

if not os.path.exists("particle_data"):os.makedirs("particle_data")

Parameters

In [ ]:
INT_RAD = 1.0 # Radius of interior particle
DOMAIN_TO_RAD = 15 # Ratio of domain radius to interior particle radius
INT_DIA = 2.0*INT_RAD  # Diameter of interior particle
DOMAIN_RAD = DOMAIN_TO_RAD * INT_RAD  #  Radius of the domain

In [ ]:
part_rad = INT_RAD
cell_width = 4*part_rad
nn_cell_width = 4*part_rad
sq_cutoff_rad = (nn_cell_width)**2
dt = 0.01

In [ ]:
final_episode = 50 # Number of times to repeat the test run
print_episode = 0 

Declare C function prototypes for "mixing.c"

In [ ]:
# Load the shared library
path = os.getcwd()
mixing_lib = ctypes.CDLL(os.path.join(path,'mixing.so')) # '.so' is the compiled C file name

# Define the C function signature
process = mixing_lib.mix_state
process.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double),\
                    ctypes.c_int, ctypes.c_int, ctypes.c_double, ctypes.c_double,\
                    ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_double]#, ctypes.c_double, ctypes.c_double]
#state, thetaRL, rows, cols, P_rad, D_rad, N_wall, N_passive, N_active, delta_t, Time
process.restype = ctypes.POINTER(ctypes.c_double)

# To free the dynamic array in C
dealloc_mem = mixing_lib.free_array
dealloc_mem.argtypes = [ctypes.POINTER(ctypes.c_double)]

Function call for mixing process

In [ ]:
def call_mix_function(state_array, theta_array, del_t):
    rows = state_array.shape[0]
    cols = state_array.shape[1]
    
    # Convert the state to a flat C-style array
    state_array = state_array.astype(np.float64)
    state_array_flat = state_array.flatten(order='C')
    cstyle_state = state_array_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    
    # Convert theta to a C-style array
    theta_array = theta_array.astype(np.float64)
    cstyle_theta = theta_array.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

    #Passing arguements to C function.................................................
    new_state = process(cstyle_state, cstyle_theta, rows, cols, INT_RAD, DOMAIN_RAD,\
                        N_WALL, N_PASSIVE, N_ACTIVE, del_t)#,Time)
    #...................................................................................
    
    # Convert the pointer to a Python array
    python_state = np.ctypeslib.as_array(new_state, shape=(rows * cols,))
    
    # convert from 1D to 2D Python array
    mod_state = python_state.reshape(rows,cols).copy()
    
    # Free the allocated memory in C
    dealloc_mem(new_state)
    
    return mod_state

Function for Mixing Index Calculation

In [ ]:
def nn_fn(data_passive):
    
    nn_list = np.zeros(N_PASSIVE,dtype=int)
    posx_nn = data_passive[:,0]
    posy_nn = data_passive[:,1]
    head_nn = np.ones(N_cells_nn)*(-1)
    cell_x_nn = (np.divide(np.subtract(posx_nn,x_low_nn),nn_cell_width)).astype(int) # Putting agent into respective xcell
    cell_y_nn = (np.divide(np.subtract(posy_nn,y_low_nn),nn_cell_width)).astype(int); # Putting agent into respective ycell
    cell_pos_nn = (cell_x_nn*Nc_y_nn + cell_y_nn).astype(int); # Cell number calculated vertically
    
    #assigning particles' (identity) in head2 & nn_list for easily accessing
    for i in range (N_PASSIVE):
        nn_list[i] = head_nn[cell_pos_nn[i]]
        head_nn[cell_pos_nn[i]] = i
        
    pass_type = np.zeros((N_PASSIVE,2))

    for i in range (Nc_x_nn):
        for j in range (Nc_y_nn):
            nn_cell_n = i*Nc_y_nn + j
            iat = int(head_nn[nn_cell_n])
            while (iat != -1): # -1 implies no particles in this head/cell 
                for neigh_cell_x in [i-1,i,i+1]:
                    for neigh_cell_y in [j-1,j,j+1]:
                        neigh_cell_n = neigh_cell_x*Nc_y_nn + neigh_cell_y
                        jat = int(head_nn[neigh_cell_n])
                        while (jat != -1): # -1 implies no particles in this head/cell 
                            if(iat != jat):
                                # Distance calculation of neighboring particles
                                dx = posx_nn[jat] - posx_nn[iat]
                                dy = posy_nn[jat] - posy_nn[iat]
                                sq_distance = dx*dx + dy*dy
                                
                                if (sq_distance <=sq_cutoff_rad):
                                    if(jat<PASSIVE_HALF):
                                        pass_type[iat][0] += 1 # TYPE 1 PASSIVE
                                    if(jat>=PASSIVE_HALF):
                                        pass_type[iat][1] += 1 # TYPE 2 PASSIVE

                            jat = int(nn_list[jat])

                iat = nn_list[iat]
    #print(np.shape(pass_type))
    ratio_1 = np.sum(2*np.nan_to_num(np.divide(pass_type[:PASSIVE_HALF,1],(pass_type[:PASSIVE_HALF,0]+pass_type[:PASSIVE_HALF,1]))),axis=0)
    ratio_2 = np.sum(2*np.nan_to_num(np.divide(pass_type[-PASSIVE_HALF:,0],(pass_type[-PASSIVE_HALF:,0]+pass_type[-PASSIVE_HALF:,1]))),axis=0)
    mix_index = (ratio_1+ratio_2)/N_PASSIVE
#     print(mix_index)
    return mix_index

Functions in class Simulate

In [ ]:
class Simulate:

    """To read initial particle positions and return passive and active state (x,y,u,v) separately"""
    @staticmethod
    def interiorParticlesRead():
        global theta, rad, vel, part_type,  N_WALL, N_PASSIVE, N_ACTIVE,\
            INT_PARTICLES, N_TOTAL, PASSIVE_HALF, x_low, y_low, x_max, y_max, Nc_x, Nc_y, N_cells,\
            x_low_nn, y_low_nn, x_max_nn, y_max_nn, Nc_x_nn, Nc_y_nn, N_cells_nn
        part = pd.read_csv("ini_state_q1.csv", header = 0, delimiter="\t", names=["x", "y",\
                        "radius", "part_type"])#Read values and assigned under these headers
        pos_x = np.asarray(part['x']) #Converts the values under x to an array named x
        pos_y = np.asarray(part['y']) #Converts the values under y to an array named y
        rad = np.asarray(part['radius']).reshape(-1,1)
        part_type = np.asarray(part['part_type']).reshape(-1,1) #Type of particle : 1-interior , 0-boundary
        
        x_low = np.amin(pos_x)-cell_width
        y_low = np.amin(pos_y)-cell_width
        x_max = np.amax(pos_x)+cell_width
        y_max = np.amax(pos_y)+cell_width
        Nc_x = int(np.ceil((x_max-x_low)/cell_width))
        Nc_y = int(np.ceil((y_max-y_low)/cell_width))
        N_cells = int(Nc_x*Nc_y) # Total no of cells
        
        x_low_nn = np.amin(pos_x)-nn_cell_width
        y_low_nn = np.amin(pos_y)-nn_cell_width
        x_max_nn = np.amax(pos_x)+nn_cell_width
        y_max_nn = np.amax(pos_y)+nn_cell_width
        Nc_x_nn = int(np.ceil((x_max_nn-x_low_nn)/nn_cell_width))
        Nc_y_nn = int(np.ceil((y_max_nn-y_low_nn)/nn_cell_width))
        N_cells_nn = int(Nc_x_nn*Nc_y_nn) # Total no of cells
        
        N_WALL = len((part_type[part_type==0]))
        N_PASSIVE = len((part_type[part_type==-1]))
        N_ACTIVE = len((part_type[part_type==1]))
        INT_PARTICLES = N_ACTIVE + N_PASSIVE
        N_TOTAL = INT_PARTICLES + N_WALL
        PASSIVE_HALF = int(N_PASSIVE/2)
        del part # Memory freed
    
    '''Initial state : (x, y) of passive & active particles'''
    def readInitialState():
        """
        Ensure all data is entered through a single file in the format given below.
        Ensure the data is in order of wall data, passive data and active data 
        from top to bottom.The initial input file contained u,v velocities from which 
        orientation (theta) is calculated.
        """
        #active, passive, and wall data : x, y, rad, vel, theta, type : indices :0 1 2 3 4 5
        global wall_state
        part = pd.read_csv("ini_state_q1.csv", delimiter="\t",header=0) 
        initial_state = part.to_numpy(dtype=float)
        int_state = initial_state[:INT_PARTICLES, :2] # Storing initial position of active particles of active particles
        wall_state = initial_state[INT_PARTICLES:, :2] # Storing wall particle x,y data
        del part
        return int_state # Returns initial state : (x, y) of passive & active particles    
    
    '''Receives a state, does dynamics through a series of sub functions and returns new state'''
    @staticmethod
    def systemState(interior_state, theta_RL, delta_t, iteration): # receives (x, y), theta(RL) data - for active+passive separate
        global wall_state, rad, part_type
        state = np.concatenate((interior_state,wall_state), axis=0) # active+passive+wall - x,y data
        theta_up = np.concatenate((theta_RL, np.zeros(N_PASSIVE) , np.zeros(N_WALL)),axis=0).reshape(-1,1)
        c_state = np.hstack((state, rad, part_type)) # x,y,rad,part_type for all particles

        #......................................................
        up_state = call_mix_function(c_state,theta_up,delta_t) # CALLING FUNCTION FOR MIXING
        #......................................................
                    
        #if print_episode==final_episode: # printing the state variables for the final episode in valuation
        if print_episode%1==0:# printing the state variables for every episode
            if iteration%1000 == 0: # printing after "iteration*dt" seconds if condition is true
                outfile = "pos_data/RL_particle_data_{:d}/state_{:.2f}".format(print_episode,iteration*dt+delta_t) + ".csv"
                df = pd.DataFrame(np.hstack((up_state[:,:2], rad.reshape(-1,1), theta_up.reshape(-1,1), part_type.reshape(-1,1))),\
                                  columns=['x','y','r','theta','part'])
                df.to_csv(outfile, index=False, sep='\t')
         
        '''rew is the reward or mixing index based on current position of passive particles'''
        rew = nn_fn(up_state[N_ACTIVE:INT_PARTICLES,:2])
                
        obs_state = up_state[:INT_PARTICLES, :2]
        return obs_state, rew # Returning passive, active states separately along 
                                                      # with combined index after delta_t iterations (say 1 sec)

In [ ]:
Simulate.interiorParticlesRead()

# RL Implementation

In [ ]:
import gym
import tqdm
import rich
from gym import spaces
from stable_baselines3.common.env_checker import check_env
# from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import tensorflow as tf
import datetime, os

In [ ]:
# Create directory particle_data for storing data
for i in range (int(final_episode)+1):
    os.makedirs("pos_data/RL_particle_data_{:d}".format(i))

Directories for Saving

In [ ]:
models_dir = "models/PPO"
log_dir =  "logs"

if not os.path.exists("RL_particle_data"):  # Create directory particle_data for storing data
  os.makedirs("RL_particle_data")

if not os.path.exists(models_dir):  # Create directory particle_data for storing data
  os.makedirs(models_dir)
if not os.path.exists(log_dir):  # Create directory particle_data for storing data
  os.makedirs(log_dir)

In [ ]:
sim_time = 50000.0 # Total Simulation Time

Environment

In [ ]:
'''t_sim determines the time duration for an episode'''
'''delta_t is the interval with in which RL interacts with direct run'''

class ParticleMixer(gym.Env):
    """Custom Environment that follows gym interface"""
    
    def __init__(self, n_dim=2, t_sim=sim_time, delta_t=20.0):  # self is a constructor
        super(ParticleMixer, self).__init__()
        self.n_obs = INT_PARTICLES  # Observing particle count
        self.n_active = N_ACTIVE
        
        # Define the pool of angles
        self.angle_pool = np.arange(0, 2.0*pi, pi/2) # action space is (1,2,3,......,360)*pi/180 radians

        # Setting up the action space                                                                                                            
        self.action_space = gym.spaces.MultiDiscrete([len(self.angle_pool)] * self.n_active)

        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=2*DOMAIN_RAD,
                                            shape=((self.n_obs)*n_dim, ), dtype=np.float64)  # Observation space size: [n_passive+n_active] x 2 : (x,y)                                    

        self.obs_state = np.zeros((self.n_obs,2), dtype=np.float64) # 2 columns (x,y)
        self.t_sim = t_sim
        self.n_time = 0.0
        self.del_t = delta_t
        self.t = 0.0
        self.prev_action = self.action_space.sample() # creates a sample action space
        self.done = False

    def step(self, action): #This function is called by alg after reset function to perform the action
                            # This function is linked to the system dynamics equation
        self.action_val = [self.angle_pool[i] for i in action]  # Sampled angles for active particles
        #print(self.prev_action)
        
        """ To find & assign velocity when angle "d_theta" is the action_space """

        #self.active_state[:,2] = self.prev_action  # theta assigned

        self.obs_state, index = Simulate.systemState(self.obs_state, self.action_val, self.del_t, self.n_time) # sending x, y, theta
        """ Send current [position, theta] and returns new [position, theta] after implementing Langevin code for delta_t time"""
        
        '''We take the -ve of index as reward. As mixing is better, index value decreases
        If we take the negative, the value can be seen increasing with mixing (in the -ve range towards 0)
        As the algorithm tends to maximise reward, the cumulative reward should be closer to 0'''
        reward = index 
        observation = self.obs_state.flatten() # self.observation_space.sample()
        info = {}
        self.t += self.del_t  # Total Time Completed
        self.n_time += self.del_t/0.01  # Total Timesteps completed
        if self.t >= self.t_sim: self.done = True
        return observation, reward, self.done, info

    def reset(self): #While running the RL algorithm, this function is called first. This sets initial observation space
        self.t = 0.0
        self.n_time = 0.0
        self.done=False     
        self.obs_state = Simulate.readInitialState() # Resetting the initial state before an episode during first case : (x, y, theta(from i/p file))
        observation = self.obs_state.flatten() # 1-D initial state
#         print("1 reset")
        return observation  # reward, done, info can't be included
    def render(self):
        pass
    def close (self):
        pass

Assigning environment to variable env

In [ ]:
env = ParticleMixer() 

Defining the arrays to save the mixing indices

In [ ]:
MI=np.zeros([int(env.t_sim/env.del_t),final_episode]) 
#use int(env.t_sim/env.del_t+1) if division yields non-zero remainder

Check the custom environment for errors

In [ ]:
check_env(env);

Algorithm and Policy

In [ ]:
from stable_baselines3 import PPO

In [ ]:
# for bypassing errors caused by Gym version differences when restoring environment objects.
# Use custom_objects to ignore the problematic object
model_ppo = PPO.load("250.zip", env=env, custom_objects={"action_space": env.action_space,
                    "observation_space":env.observation_space})

Test Run using Learned Model

In [ ]:
sim_start = time.time()
n_episodes = final_episode
l=0
condition=0
for episode in range(n_episodes):
    print_episode=episode+1
    state_up = env.reset()
    done = False
    score = 0
    m=0
    while not done:
        action,_ = model_ppo.predict(state_up) #Predicting the actions based on the trained model
        #action,_ = model_a2c.predict(state_up)
        state_up, reward, done, info = env.step(action)
        score += reward
        if env.n_time % 100000 == 0: 
          print(f"Episode {episode+1}: t={env.t:.2f}, reward={reward:.2f}")
        MI[m][l]=reward
        m=m+1
        
    print(f"Episode: {episode+1} has score: {score}")
    l=l+1
    
TIM = np.arange(env.del_t,sim_time+1,env.del_t).reshape(-1,1)
np.savetxt('MI_vs_Time.csv',np.column_stack((TIM , MI)), delimiter='\t') # Saving the reward for every episode
sim_learn_time = time.time()-sim_start


In [ ]:
TIM = np.arange(env.del_t,sim_time+1,env.del_t).reshape(-1,1)
np.savetxt('MI_vs_Time.csv',np.column_stack((TIM , MI)), delimiter='\t') # Saving the reward for every episode
sim_learn_time = time.time()-sim_start